In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1  opencv-python matplotlib

In [2]:
!pip install tensorflow-hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt
import cv2


In [4]:
from tqdm import tqdm
import time

# Simulating package download using a loop
total_packages = 10
for package_number in tqdm(range(total_packages), desc="Downloading packages"):
    # Simulating download delay
    time.sleep(0.5)  # Remove this line and replace with actual download code


LOAD MODEL

In [5]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

MAKE DETECTIONS

In [6]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()    #return value,image
    
    #reshape image cuz Movenet expects input to be 192x192x3 and float32
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,256)
    input_img = tf.cast(img,dtype=tf.int32)
    
    
    #Detections
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    print(keypoints_with_scores)
   
    
    
    
    #Rendering
    loop_through_people(frame,keypoints_with_scores,EDGES,0.3)
   
    
    cv2.imshow('MoveNet Multipose',frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[[6.15824699e-01 8.30269992e-01 1.80919603e-01]
  [5.03721356e-01 8.48102450e-01 4.53501791e-01]
  [5.18016458e-01 7.78070033e-01 1.86308593e-01]
  [5.29927135e-01 7.98005164e-01 2.72204161e-01]
  [5.11388242e-01 5.70546687e-01 4.53284800e-01]
  [8.86852205e-01 8.31243455e-01 3.94961059e-01]
  [8.76189888e-01 3.59311998e-01 1.83574751e-01]
  [9.39131975e-01 8.24714303e-01 9.66103654e-03]
  [9.80605364e-01 1.53028756e-01 4.05394734e-04]
  [7.55504787e-01 7.63690174e-01 1.61266383e-02]
  [9.31329370e-01 4.62064326e-01 1.36426901e-02]
  [9.81241465e-01 5.86082399e-01 3.16215755e-06]
  [9.55571175e-01 3.43351156e-01 1.75889523e-04]
  [9.40991163e-01 7.18399048e-01 1.79744978e-02]
  [9.55071390e-01 3.69687021e-01 4.22572941e-02]
  [4.57243145e-01 3.94128799e-01 1.76719623e-03]
  [5.23826122e-01 5.55865109e-01 3.78306489e-03]]

 [[3.92224848e-01 1.01505496e-01 1.31873181e-03]
  [3.72020334e-01 8.20977315e-02 6.05518813e-04]
  [3.84098381e-01 8.80141780e-02 6.37661142e-04]
  [3.59092772e-01 

NameError: name 'loop_through_people' is not defined

In [ ]:
results   # y*x*score = 3  =3*17keypoints 

#3*17 +5-->these 5 are bounding box points so total its 56keypoints

In [ ]:
results['output_0'].numpy()[:,:,:51]  #only body keypoints.. for all of the 6 people

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

In [ ]:
np.array(left_elbow[:2]*[480,640]).astype(int)

In [ ]:
np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']),[480,640,1]))

Draw Keypoints

In [ ]:
def draw_keypoints(frame,keypoints,confidence_threshold):
    y,x,c = frame.shape    #confidence/prob - c
    shaped = np.squeeze(np.multiply(keypoints,[y,x,1]))   #multiplies keypoints and y,x,1 
    
    for kp in shaped:
        ky,kx,kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame,(int(kx),int(ky)),4,(0,255,0),-1)   #color (0,255,0) BGR  #-1 indicates it fills in with that color mentioned
        

In [ ]:
def loop_through_people(frame,keypoint_with_scores,edges,confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame,person,edges,confidence_threshold)
        draw_keypoints(frame,person,confidence_threshold)     #looping to all 6 ppl in frame
        

In [ ]:
keypoints_with_scores.shape    #returns 6 ppl,17keypoints, y,x,c

Draw Edges

In [ ]:
EDGES = {
       (0,1):'m',
       (0,2):'c',
       (1,3):'m',
       (2,4):'c',
       (0,5):'m',
       (0,6):'c',
       (5,7):'m',
       (7,9):'m',
       (6,8):'c',
       (8,10):'c',
       (5,6):'y',
       (5,11):'m',
       (6,12):'c',
       (11,12):'y',
       (11,13):'m',
       (13,15):'m',
       (12,14):'c',
       (14,16):'c'
}

# edge:color  for connections btw diff keypoint for eg point 0 connectws to 1 ...refer movenet doc for respective keypoints ref to body

In [ ]:
def draw_connections(frame,keypoints,edges,confidence_threshold):
    y,x,c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints,[y,x,1]))
    
    for edge, color in EDGES.items():
        p1,p2 = edge
        y1,x1,c1 = shaped[p1]
        y2,x2,c2 = shaped[p2]
        print(y1,x1,c1,y2,x2,c2)
        
        if (c1 > confidence_threshold) & (c2> confidence_threshold):
            cv2.line(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,255),2)

In [ ]:
cap = cv2.VideoCapture('"Put your video sample here"')
while cap.isOpened():
    ret,frame = cap.read()    #return value,image
    
    #reshape image cuz Movenet expects input to be 192x192x3 and float32
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),352,640)
    input_img = tf.cast(img,dtype=tf.int32)
    
    
    #Detections
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    print(keypoints_with_scores)
   
  
    
    #Rendering
    loop_through_people(frame,keypoints_with_scores,EDGES,0.3)
   
    
    cv2.imshow('MoveNet Multipose',frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#For video rescaling
frame.shape

In [ ]:
#Resize accord to vid for movenet model it expects longer side of vid to be min 256 and others to be divisble by 32
1080/1920

In [ ]:
1920/3

In [ ]:
640*0.5625

In [ ]:
360/32

In [ ]:
#So close to 11.25 then 32*11 = 352 scale down

In [7]:
cap.release()
cv2.destroyAllWindows()